## Downloading Dependencies

In [1]:
# First, we need to install pycocotools. This library will be used for computing the evaluation metrics following the COCO metric for intersection over union.
# The version by default in Colab has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
!pip install cython
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-idwf7nw_
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-idwf7nw_
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=265187 sha256=de26d45b4bacbc82780afd1444461ddb7e3c948d0074892a498aa1a0cb050ca2
  Stored in directory: /tmp/pip-ephem-wheel-cache-un3je0fs/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


In [2]:
import json
import math
import os
import random
import sys
import time

import numpy as np
import torch
import torch.utils.data
import torchvision

from PIL import Image, ImageDraw
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision import transforms
from torchvision.ops import batched_nms
from torchvision.transforms import functional as F

In [3]:
from typing import List, Tuple

import torchvision.models as models

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.faster_rcnn import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import MultiScaleRoIAlign

In `references/detection/,` we have a number of helper functions to simplify training and evaluating detection models.
Here, we will use `references/detection/utils.py` and `references/detection/coco_eval.py`.

Let's copy those files (and their dependencies) in here so that they are available in the notebook

In [4]:
%%shell

# Download TorchVision repo to use some files from references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/coco_eval.py ../

Cloning into 'vision'...
remote: Enumerating objects: 153211, done.
remote: Counting objects: 100% (5584/5584), done.
remote: Compressing objects: 100% (524/524), done.
remote: Total 153211 (delta 5085), reused 5475 (delta 5037), pack-reused 147627
Receiving objects: 100% (153211/153211), 302.29 MiB | 17.43 MiB/s, done.
Resolving deltas: 100% (135664/135664), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be376084d version check against PyTorch's CUDA version


In [5]:
import utils
from coco_eval import CocoEvaluator

In [6]:
# check if cuda GPU is available, make sure you're using GPU runtime on Google Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # you should output "cuda"

cuda


In [7]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
! pip install ensemble-boxes


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


That's all for the dataset. Let's see how the outputs are structured for this dataset

In [9]:
# hyper-parameters
params = {'BATCH_SIZE': 32,
          'LR': 1e-4,
          'CLASSES': 2+1,
          'MAXEPOCHS': 5,
          'BACKBONE': 'resnet50',
          'FPN': True,
          # 'ANCHOR_SIZE': ((32,), (64,), (128,), (256,), (512,)),
          'ANCHOR_SIZE': ((256,), (16,), (32,), (64,), (128,)),
          'ASPECT_RATIOS': ((0.5, 1.0, 2.0),),
          'MIN_SIZE': 256,
          'MAX_SIZE': 256,
          'IMG_MEAN': [0.485, 0.456, 0.406],
          'IMG_STD': [0.229, 0.224, 0.225],
          'IOU_THRESHOLD': 0.5
          }

In [10]:

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
num_classes = 3

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)  # ,trainable_backbone_layers=1
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [11]:
retina_model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=False)
num_classes = 3
in_features = retina_model.head.classification_head.conv[0].in_channels
num_anchors = retina_model.head.classification_head.num_anchors
retina_model.head.classification_head.num_classes = num_classes

cls_logits = torch.nn.Conv2d(in_features, num_anchors * num_classes, kernel_size = 3, stride=1, padding=1)
torch.nn.init.normal_(cls_logits.weight, std=0.01)  # as per pytorch code
torch.nn.init.constant_(cls_logits.bias, -math.log((1 - 0.01) / 0.01))  # as per pytorcch code 
# assign cls head to model
retina_model.head.classification_head.cls_logits = cls_logits

In [12]:
fcos_model = torchvision.models.detection.fcos_resnet50_fpn(pretrained=False)
fcos_model.head.classification_head.cls_logits.out_channels = 3
fcos_model.transform.max_size = 800

## Generate Predictions on Test Images

In [13]:
til_test_root = "/content/drive/MyDrive/Qualifiers/CV Final Evaluation Dataset/" # extracted testing images path
test_img_root = os.path.join(til_test_root, "Images")
# img_dir = os.scandir(test_img_root)
test_annotation = os.path.join(til_test_root, 'qualifiers_finals_no_annotations.json')

In [14]:
# load model weights (if not using the current trained model)
# save_path = '/content/drive/MyDrive/model_weights/cv/retina_06-03-2022-02:07:05_e4'  
# save_path = '/content/drive/MyDrive/model_weights/cv/retina_06-04-2022-02:48:01_e7' 
save_path = '/content/drive/MyDrive/model_weights/cv/retina_06-04-2022-14:42:42_e7' # uSE THIS

model_name = save_path.split('/')[-1] # use same as model name for easy tracking
retina_model.load_state_dict(torch.load(save_path, map_location=device))
retina_model.to(device)
retina_model.eval()
model_name

'retina_06-04-2022-14:42:42_e7'

In [15]:
# load model weights (if not using the current trained model)
# save_path = '/content/drive/MyDrive/model_weights/cv/frcnn_06-05-2022-01:10:28_e9'
save_path = '/content/drive/MyDrive/model_weights/cv/frcnn_pt_06-08-2022-03:28:07_e3'  # USE THIS
# save_path = '/content/drive/MyDrive/model_weights/cv/frcnn_pt_06-07-2022-23:34:40_e1'
model_name = save_path.split('/')[-1] # use same as model name for easy tracking
model.load_state_dict(torch.load(save_path, map_location=device))
model.to(device)
model.eval()
model_name

'frcnn_pt_06-08-2022-03:28:07_e3'

In [16]:
# load model weights (if not using the current trained model)
save_path = '/content/drive/MyDrive/model_weights/cv/fcos_06-06-2022-01:36:33_e9'  # USE THIS
# save_path = '/content/drive/MyDrive/model_weights/cv/fcos_06-08-2022-01:22:59_e4'  # try this

model_name = save_path.split('/')[-1] # use same as model name for easy tracking
fcos_model.load_state_dict(torch.load(save_path, map_location=device))
fcos_model.to(device)
fcos_model.eval()
model_name

'fcos_06-06-2022-01:36:33_e9'

In [17]:
with open(test_annotation, 'r') as f:
  test_images = json.load(f)

Let's visualize some predictions on the test images. Run this a few times to visualize different images.

In [18]:
CAT_2_NAME = {1: 'Fallen', 2: 'Standing'}

In [19]:
from ensemble_boxes import *

def normalize_box(bbox_preds, h, w):
  for x1, y1, x2, y2 in bbox_preds:
    x1 /= w
    x2 /= w
    y1 /= h
    y2 /= h
  return bbox_preds

In [21]:
# visualize image (just frcnn)

img_path = os.path.join(test_img_root, test_images['images'][1900]['file_name']) # 637 
# img_path = next(img_dir).path
img = Image.open(img_path).convert('RGB')
draw = ImageDraw.Draw(img)
det_threshold = 0.

# do the prediction
with torch.no_grad():
    img_tensor = transforms.ToTensor()(img)
    img_preds = model([img_tensor.to(device)])[0]

for i in range(len(img_preds["boxes"])):
    x1, y1, x2, y2 = img_preds["boxes"][i]
    label = int(img_preds["labels"][i])
    score = float(img_preds["scores"][i])

    # filter out non-confident detections
    if score > det_threshold:
        draw.rectangle(((x1, y1), (x2, y2)), outline="red")
        text = f'{CAT_2_NAME[label]}: {score}'
        draw.text((x1+5, y1+5), text)

display(img)

# TODO: find bad cases in test set
# TODO: save pred images in drive

Output hidden; open in https://colab.research.google.com to view.

In [20]:
def reverse_horizontal_flip(bbox, height, width):
  bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
  return bbox

In [23]:
# visualize image with WBF

img_path = os.path.join(test_img_root, test_images['images'][0]['file_name']) # 637 
# img_path = next(img_dir).path
img = Image.open(img_path).convert('RGB')
draw = ImageDraw.Draw(img)
det_threshold = 0.  

# do the prediction
with torch.no_grad():
    img_tensor = transforms.ToTensor()(img)
    img_preds = model([img_tensor.to(device)])[0]
    h, w = img_tensor.shape[1], img_tensor.shape[2]

    img_tensor_gray = transforms.Compose([
                          transforms.Grayscale(),
                          transforms.ToTensor()
                      ])(img)
    img_tensor_blur = transforms.Compose([
                          transforms.GaussianBlur(kernel_size=9),
                          transforms.ToTensor()
                      ])(img)
    img_tensor_flip = transforms.Compose([
                          transforms.RandomHorizontalFlip(1),
                          transforms.ToTensor(),
    ])(img)

    # fasterrcnn model
    img_preds = model([img_tensor.to(device)])[0]
    gray_preds = model([img_tensor_gray.to(device)])[0]
    blur_preds = model([img_tensor_blur.to(device)])[0]

    # retina model
    img_preds_retina = retina_model([img_tensor.to(device)])[0]
    gray_preds_retina = retina_model([img_tensor_gray.to(device)])[0]
    blur_preds_retina = retina_model([img_tensor_blur.to(device)])[0]   

    # fcos model
    img_preds_fcos = fcos_model([img_tensor.to(device)])[0]
    gray_preds_fcos = fcos_model([img_tensor_gray.to(device)])[0]
    blur_preds_fcos = fcos_model([img_tensor_blur.to(device)])[0]   


    flip_preds = model([img_tensor_flip.to(device)])[0]
    flip_preds_retina = retina_model([img_tensor_flip.to(device)])[0]
    flip_preds_fcos = fcos_model([img_tensor_flip.to(device)])[0]

    flip_preds['boxes'] = reverse_horizontal_flip(flip_preds['boxes'], h, w)
    flip_preds_retina['boxes'] = reverse_horizontal_flip(flip_preds_retina['boxes'], h, w)
    flip_preds_fcos['boxes'] = reverse_horizontal_flip(flip_preds_fcos['boxes'], h, w)

    img_preds['boxes'] = normalize_box(img_preds['boxes'], h, w)
    gray_preds['boxes'] = normalize_box(gray_preds['boxes'], h, w)
    blur_preds['boxes'] = normalize_box(blur_preds['boxes'], h, w)
    flip_preds['boxes'] = normalize_box(flip_preds['boxes'], h, w)

    img_preds_retina['boxes'] = normalize_box(img_preds_retina['boxes'], h, w)
    gray_preds_retina['boxes'] = normalize_box(gray_preds_retina['boxes'], h, w)
    blur_preds_retina['boxes'] = normalize_box(blur_preds_retina['boxes'], h, w)
    flip_preds_retina['boxes'] = normalize_box(flip_preds_retina['boxes'], h, w)

    img_preds_fcos['boxes'] = normalize_box(img_preds_fcos['boxes'], h, w)
    gray_preds_fcos['boxes'] = normalize_box(gray_preds_fcos['boxes'], h, w)
    blur_preds_fcos['boxes'] = normalize_box(blur_preds_fcos['boxes'], h, w)
    flip_preds_fcos['boxes'] = normalize_box(flip_preds_fcos['boxes'], h, w)

    bbox_list = [
      img_preds['boxes'].tolist(),
      gray_preds['boxes'].tolist(),
      blur_preds['boxes'].tolist(),
      flip_preds['boxes'].tolist(),

      img_preds_retina['boxes'].tolist(),
      gray_preds_retina['boxes'].tolist(),
      blur_preds_retina['boxes'].tolist(),
      flip_preds_retina['boxes'].tolist(),


      img_preds_fcos['boxes'].tolist(),
      gray_preds_fcos['boxes'].tolist(),
      blur_preds_fcos['boxes'].tolist(),
      flip_preds_fcos['boxes'].tolist(),
    ]
    scores_list = [
      img_preds['scores'].tolist(),
      gray_preds['scores'].tolist(),
      blur_preds['scores'].tolist(),  
      flip_preds['scores'].tolist(),

      img_preds_retina['scores'].tolist(),
      gray_preds_retina['scores'].tolist(),
      blur_preds_retina['scores'].tolist(),  
      flip_preds_retina['scores'].tolist(),

      img_preds_fcos['scores'].tolist(),
      gray_preds_fcos['scores'].tolist(),
      blur_preds_fcos['scores'].tolist(),  
      flip_preds_fcos['scores'].tolist(),

    ]
    labels_list = [
      img_preds['labels'].tolist(),
      gray_preds['labels'].tolist(),
      blur_preds['labels'].tolist(),  
      flip_preds['labels'].tolist(),

      img_preds_retina['labels'].tolist(),
      gray_preds_retina['labels'].tolist(),
      blur_preds_retina['labels'].tolist(),  
      flip_preds_retina['labels'].tolist(),

      img_preds_fcos['labels'].tolist(),
      gray_preds_fcos['labels'].tolist(),
      blur_preds_fcos['labels'].tolist(),  
      flip_preds_fcos['labels'].tolist(),
    ]

    boxes, scores, labels = weighted_boxes_fusion(bbox_list,scores_list, labels_list,weights=[1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1], iou_thr=0.5, skip_box_thr=0.01)
    for idx, (x1, y1, x2, y2) in enumerate(boxes):
        boxes[idx] = [x1*w, y1*h, x2*w, y2*h]

    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        label = int(labels[i])
        score = float(scores[i])
        x1 = max(int(x1), 0)
        y1 = max(int(y1), 0)
        x2 = min(int(x2), w)
        y2 = min(int(y2), h)

        left = int(x1)
        top = int(y1)
        width = int(x2 - x1)
        height = int(y2 - y1)

        # filter out non-confident detections
        if score > det_threshold:
            draw.rectangle(((x1, y1), (x2, y2)), outline="red")
            text = f'{CAT_2_NAME[label]}: {score}'
            draw.text((x1+5, y1+5), text)

display(img)


Output hidden; open in https://colab.research.google.com to view.

## Submission of Results

Submission json file should be in [COCO format](https://cocodataset.org/#format-results).

```
[{
    "image_id": int, 
    "category_id": int, 
    "bbox": [x,y,width,height], 
    "score": float,
}]
```

Refer to **sample_submission_cv.json** for an example.

For this competition, the metric for evaluation will be mAP @ 0.50:0.95

In [ ]:
# using WBF ensemble (frcnn and retina, with weights 1 and 2 respectively)
from tqdm.notebook import tqdm

detections = []
with torch.no_grad():
    for image in tqdm(test_images['images'], total=len(test_images['images'])):
        img_id = int(image['id'])
        img = Image.open(os.path.join(test_img_root, image['file_name'])).convert('RGB')
        img_tensor = transforms.ToTensor()(img)
        img_preds = model([img_tensor.to(device)])[0]
        h, w = img_tensor.shape[1], img_tensor.shape[2]

        # img_tensor_gray = transforms.Compose([
        #                       transforms.Grayscale(),
        #                       transforms.ToTensor()
        #                   ])(img)
        img_tensor_blur = transforms.Compose([
                              transforms.GaussianBlur(kernel_size=9),
                              transforms.ToTensor()
                          ])(img)
        img_tensor_flip = transforms.Compose([
                              transforms.RandomHorizontalFlip(1),
                              transforms.ToTensor(),
        ])(img)

        # fasterrcnn model
        img_preds = model([img_tensor.to(device)])[0]
        # gray_preds = model([img_tensor_gray.to(device)])[0]
        blur_preds = model([img_tensor_blur.to(device)])[0]

        # retina model
        img_preds_retina = retina_model([img_tensor.to(device)])[0]
        # gray_preds_retina = retina_model([img_tensor_gray.to(device)])[0]
        blur_preds_retina = retina_model([img_tensor_blur.to(device)])[0]   

        # fcos model
        img_preds_fcos = fcos_model([img_tensor.to(device)])[0]
        # gray_preds_fcos = fcos_model([img_tensor_gray.to(device)])[0]
        blur_preds_fcos = fcos_model([img_tensor_blur.to(device)])[0]   


        flip_preds = model([img_tensor_flip.to(device)])[0]
        flip_preds_retina = retina_model([img_tensor_flip.to(device)])[0]
        flip_preds_fcos = fcos_model([img_tensor_flip.to(device)])[0]

        flip_preds['boxes'] = reverse_horizontal_flip(flip_preds['boxes'], h, w)
        flip_preds_retina['boxes'] = reverse_horizontal_flip(flip_preds_retina['boxes'], h, w)
        flip_preds_fcos['boxes'] = reverse_horizontal_flip(flip_preds_fcos['boxes'], h, w)

        img_preds['boxes'] = normalize_box(img_preds['boxes'], h, w)
        # gray_preds['boxes'] = normalize_box(gray_preds['boxes'], h, w)
        blur_preds['boxes'] = normalize_box(blur_preds['boxes'], h, w)
        flip_preds['boxes'] = normalize_box(flip_preds['boxes'], h, w)

        img_preds_retina['boxes'] = normalize_box(img_preds_retina['boxes'], h, w)
        # gray_preds_retina['boxes'] = normalize_box(gray_preds_retina['boxes'], h, w)
        blur_preds_retina['boxes'] = normalize_box(blur_preds_retina['boxes'], h, w)
        flip_preds_retina['boxes'] = normalize_box(flip_preds_retina['boxes'], h, w)

        img_preds_fcos['boxes'] = normalize_box(img_preds_fcos['boxes'], h, w)
        # gray_preds_fcos['boxes'] = normalize_box(gray_preds_fcos['boxes'], h, w)
        blur_preds_fcos['boxes'] = normalize_box(blur_preds_fcos['boxes'], h, w)
        flip_preds_fcos['boxes'] = normalize_box(flip_preds_fcos['boxes'], h, w)

        bbox_list = [
          img_preds['boxes'].tolist(),
          # gray_preds['boxes'].tolist(),
          blur_preds['boxes'].tolist(),
          flip_preds['boxes'].tolist(),

          img_preds_retina['boxes'].tolist(),
          # gray_preds_retina['boxes'].tolist(),
          blur_preds_retina['boxes'].tolist(),
          flip_preds_retina['boxes'].tolist(),


          img_preds_fcos['boxes'].tolist(),
          # gray_preds_fcos['boxes'].tolist(),
          blur_preds_fcos['boxes'].tolist(),
          flip_preds_fcos['boxes'].tolist(),
        ]
        scores_list = [
          img_preds['scores'].tolist(),
          # gray_preds['scores'].tolist(),
          blur_preds['scores'].tolist(),  
          flip_preds['scores'].tolist(),

          img_preds_retina['scores'].tolist(),
          # gray_preds_retina['scores'].tolist(),
          blur_preds_retina['scores'].tolist(),  
          flip_preds_retina['scores'].tolist(),

          img_preds_fcos['scores'].tolist(),
          # gray_preds_fcos['scores'].tolist(),
          blur_preds_fcos['scores'].tolist(),  
          flip_preds_fcos['scores'].tolist(),

        ]
        labels_list = [
          img_preds['labels'].tolist(),
          # gray_preds['labels'].tolist(),
          blur_preds['labels'].tolist(),  
          flip_preds['labels'].tolist(),

          img_preds_retina['labels'].tolist(),
          # gray_preds_retina['labels'].tolist(),
          blur_preds_retina['labels'].tolist(),  
          flip_preds_retina['labels'].tolist(),

          img_preds_fcos['labels'].tolist(),
          # gray_preds_fcos['labels'].tolist(),
          blur_preds_fcos['labels'].tolist(),  
          flip_preds_fcos['labels'].tolist(),
        ]
        # boxes, scores, labels = weighted_boxes_fusion(bbox_list,scores_list, labels_list,weights=[1,1,1,1, 2,2,2,2,1,1,1,1], iou_thr=0.5, skip_box_thr=0.01, conf_type='box_and_model_avg')
        boxes, scores, labels = weighted_boxes_fusion(bbox_list,scores_list, labels_list,weights=[1,1,1, 2,2,2,1,1,1], iou_thr=0.5, skip_box_thr=0.01, conf_type='box_and_model_avg')
        for idx, (x1, y1, x2, y2) in enumerate(boxes):
            boxes[idx] = [x1*w, y1*h, x2*w, y2*h]
        for i in range(len(boxes)):
            x1, y1, x2, y2 = boxes[i]
            label = int(labels[i])
            score = float(scores[i])
            if score > 1.0:
              print(score)
            x1 = max(int(x1), 0)
            y1 = max(int(y1), 0)
            x2 = min(int(x2), w)
            y2 = min(int(y2), h)

            left = int(x1)
            top = int(y1)
            width = int(x2 - x1)
            height = int(y2 - y1)

            detections.append({'image_id':img_id, 'category_id':label, 'bbox':[left, top, width, height], 'score':score})
        if len(boxes) == 0:
            print(img_id)
            detections.append({'image_id':img_id, 'category_id':1, 'bbox':[0, 0, 1, 1], 'score':0.1})
        

  0%|          | 0/1980 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:51: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:42: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')


In [ ]:
# # Using non-WBF (normal)
# from tqdm.notebook import tqdm

# detections = []
# with torch.no_grad():
#     for image in tqdm(test_images['images'], total=len(test_images['images'])):
#         img_id = int(image['id'])

#         img = Image.open(os.path.join(test_img_root, image['file_name'])).convert('RGB')
#         img_tensor = transforms.ToTensor()(img)

#         preds = model([img_tensor.to(device)])[0]

#         for i in range(len(preds["boxes"])):
#             x1, y1, x2, y2 = preds["boxes"][i]
#             label = int(preds["labels"][i])
#             score = float(preds["scores"][i])

#             left = int(x1)
#             top = int(y1)
#             width = int(x2 - x1)
#             height = int(y2 - y1)

#             detections.append({'image_id':img_id, 'category_id':label, 'bbox':[left, top, width, height], 'score':score})
#         if len(preds["boxes"]) == 0:
#             print(img_id)
#             detections.append({'image_id':img_id, 'category_id':1, 'bbox':[0, 0, 1, 1], 'score':0.1})


  0%|          | 0/1020 [00:00<?, ?it/s]

637
731


In [42]:
# test_pred_json = os.path.join('/content/drive/MyDrive/results/cv', f"test_preds_{model_name}.json")
test_pred_json = os.path.join('/content/drive/MyDrive/results/cv', f"final_eval.json")
# test_pred_json = os.path.join('/content/drive/MyDrive/results/cv', f"test_preds_ensemb_nogray.json")
# test_pred_json = os.path.join('/content/drive/MyDrive/results/cv', f"test_preds_{model_name}_sanity.json")
# test_pred_json = os.path.join('/content/drive/MyDrive/results/cv', f"test_preds_{now}_sanity.json")
with open(test_pred_json, 'w') as f:
    json.dump(detections, f)

In [43]:
len(detections)

25137